<a href="https://colab.research.google.com/github/Vitor4442/TCC-VIS-O-COMPUTACIONAL/blob/master/MipNerf360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clonando o repositorio do MIP NERF 360**

In [1]:
!git clone https://github.com/google-research/multinerf.git
%cd multinerf
!pip install -r requirements.txt

Cloning into 'multinerf'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 270 (delta 145), reused 117 (delta 117), pack-reused 88 (from 1)
Receiving objects: 100% (270/270), 174.34 KiB | 6.23 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/multinerf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00


#Alterações nas libs para se adequar ao python 3.12


In [2]:
!git clone https://github.com/rmbrualla/pycolmap.git ./internal/pycolmap
!pip install -e internal/pycolmap

Cloning into './internal/pycolmap'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 68 (delta 11), reused 7 (delta 7), pack-reused 40 (from 1)
Receiving objects: 100% (68/68), 29.58 KiB | 3.29 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Obtaining file:///content/multinerf/internal/pycolmap
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pycolmap (pyproject.toml) ... done
  Created wheel for pycolmap: filename=pycolmap-0.0.1-0.editable-py3-none-any.whl size=4540 sha256=ad8009c629c25e4a9d7b183363fb82603241d13d08eb2a234fe74736fbcb396d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qlpjiuzj/wheels/97/9b/11/900841e92a995eca0de49c0154b24c4217cd3f84569f3fcd33
Successfully built pycolmap


In [3]:
%cd /content/multinerf
!sed -i "s/np\.math\.factorial/math.factorial/g" internal/ref_utils.py
!grep -q "^import math" internal/ref_utils.py || sed -i '1iimport math' internal/ref_utils.py


/content/multinerf


In [4]:
%cd /content/multinerf
!sed -i "s/INVALID_POINT3D = np\.uint64(-1)/INVALID_POINT3D = np.iinfo(np.uint64).max/g" internal/pycolmap/pycolmap/scene_manager.py


/content/multinerf


In [5]:
%cd /content/multinerf
!sed -i "s/^import math$/import math as pymath/g" internal/ref_utils.py
!sed -i "s/math\.factorial/pymath.factorial/g" internal/ref_utils.py


/content/multinerf


#Etapa de teste

In [6]:
%cd /content/multinerf/

/content/multinerf


In [7]:
!python -m unittest discover -s tests -p "*_test.py"
#Aqui pode pular se quiser


ERROR:2026-01-17 00:07:05,256:jax._src.xla_bridge:487: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/xla_bridge.py", line 485, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 321, in initialize
    _load_nvidia_libraries()
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 133, in _load_nvidia_libraries
    _load("cublas", ["libcublas.so.12", "libcublasLt.so.12"])
  File "/usr/local/lib/python3.12/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 104, in _load
    ctypes.cdll.LoadLibrary(path)
  File "/usr/lib/python3.12/ctypes/__init__.py", line 460, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ctypes/__init__.py", line 379, in __init__
    self._hand

#Vamos iniciar o treinamento

#Instalando o dataset

In [8]:
!nvidia-smi

Sat Jan 17 00:07:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
!mkdir nerf_data

# A partir daqui eu fiz instalação do dataset no terminal mesmo
#wget http://storage.googleapis.com/gresearch/refraw360/360_v2.zip
#unzip 360_v2.zip

#Começando o treinamento

In [14]:
!python -m train \
  --gin_configs=configs/360.gin \
  --gin_bindings="Config.data_dir = '/content/multinerf/nerf_data/stump'" \
  --gin_bindings="Config.dataset_loader = 'llff'" \
  --gin_bindings="Config.checkpoint_dir = '/content/multinerf/checkpoints/360/stump_smoke'" \
  --gin_bindings="Config.batch_size = 4000" \
  --gin_bindings="Config.max_steps = 5000" \
  --logtostderr


2026-01-17 00:23:14.976281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768609394.997280    6270 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768609395.003577    6270 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768609395.019783    6270 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768609395.019833    6270 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768609395.019837    6270 computation_placer.cc:177] computation placer alr

In [15]:
!python -m render \
  --gin_configs=configs/360.gin \
  --gin_bindings="Config.data_dir = '/content/multinerf/nerf_data/stump'" \
  --gin_bindings="Config.dataset_loader = 'llff'" \
  --gin_bindings="Config.checkpoint_dir = '/content/multinerf/checkpoints/360/stump_smoke'" \
  --logtostderr


2026-01-17 01:19:35.989877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768612776.023341   20656 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768612776.033579   20656 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768612776.060538   20656 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768612776.060564   20656 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768612776.060567   20656 computation_placer.cc:177] computation placer alr

In [18]:
from IPython.display import Video, display

display(Video("/content/multinerf/checkpoints/360/stump_smoke/render/stump_smoke_360_test_preds_step_1_color.mp4", embed=True))
